# Localite Kingston Utilities Project

This notebook gives the preliminary steps in developing asset failures on an eletrical grid by leveraging machine learning.
<br> We partnered with Kingston Utilites to first assists them in identifying high-risk customer meters that can potentially fail in the future.
<br> For any further technical questions please contact Adams Liu at adamsliu97@gmail.com. 
<br> Thank you.


## Preprocessing Step 1 - Data Merging
Issue: The first 2016 service logs data is very large, each month consists of 0.5GB to a total of 12GB for one year.
       <br> Since we do not have a server I downloaded all the log files seperately and I created a new csv files that capture the total amount of KWH per month (eg. <i> KWH_Report_Jan.csv </i>)
 <br> This would make it fasters to process relevant data in the future.
    

In [2]:
#Importing Libraries

import pandas as pd
import numpy as np
from datetime import date

In [86]:
# Get the absolute difference between to dates

def days_between(aYr, aMon, aDay, bYr, bMon, bDay):
    d0 = date(aYr, aMon, aDay)
    d1 = date(bYr, bMon, bDay)
    delta = d1 - d0
    return delta.days

In [ ]:
#Creating a list of all .csv files
lsReports = ["Electric-2016-Apr.csv","electric-2016-Aug.csv","Electric-2016-Dec.csv","electric-2016-Feb.csv","electric-2016-Jan.csv","Electric-2016-Jul.csv","Electric-2016-Jun.csv",
             "electric-2016-Mar.csv","Electric-2016-May.csv","Electric-2016-Nov.csv","electric-2016-Oct.csv","electric-2016-Sep.csv"]
lsMonthes = ["Apr","Aug","Dec","Feb","Jan","Jul","Jun","Mar","May","Nov","Oct","Sep"]

#We will need to create DataFrames to be able to find so interesting datapoints 
lsMax = pd.DataFrame()
lsZeros= pd.DataFrame()
lsLess = pd.DataFrame()

In [ ]:
#We get the sum of all the monthes with unique ServiceIDs and covert them to .csv files
i = 0
for files in lsReports:
    df = pd.read_csv(files)
    df = df.sort_values(by='KWH',ascending=False)
    lsMax = lsMax.append(df.head(50))
    lsLess = lsLess.append(df[df.KWH < 0])
    lsZeros = lsZeros.append(df[df.KWH == 0])
    df2 = df[["SERVICEID","KWH"]].groupby(["SERVICEID"]).sum().to_csv("KWH_Report_" +lsMonthes[i]+".csv")
    i+=1 

#Have a seperate .csv for KWH_Max(top 50 KWH/month), KWH_LESS(all negative KWH), KWH_Zeros(all KWH that were zeros)
lsMax.to_csv("KWH_Max.csv")
lsZeros.to_csv("KWH_Zeros.csv")
lsLess.to_csv("KWH_Less.csv")

In [3]:
## Merging meters to get postal code files 
meters = pd.read_csv("meters.csv")
spCode = pd.read_csv("ServicePostalCode.csv") #there are less serviceIDs in the ServicePostalCode

meters.sort_values(by=['USDP_ID'])
spCode.sort_values(by=['service ID'])
metersMerged = pd.merge(meters, spCode, left_on = 'USDP_ID', right_on = 'service ID', how = 'left')


## Created the YYYY-MM-DD-HH attributes for the dataframe for both the start-date and end-date
metersMerged['START_YEAR'] = pd.DatetimeIndex(metersMerged['EFFECTIVE_DATE']).year
metersMerged['START_MONTH'] = pd.DatetimeIndex(metersMerged['EFFECTIVE_DATE']).month
metersMerged['START_DAY'] = pd.DatetimeIndex(metersMerged['EFFECTIVE_DATE']).day
metersMerged['START_HOUR'] = pd.DatetimeIndex(metersMerged['EFFECTIVE_DATE']).hour
metersMerged['END_YEAR'] = pd.DatetimeIndex(metersMerged['END_DATE']).year
metersMerged['END_MONTH'] = pd.DatetimeIndex(metersMerged['END_DATE']).month
metersMerged['END_DAY'] = pd.DatetimeIndex(metersMerged['END_DATE']).day
metersMerged['END_HOUR'] = pd.DatetimeIndex(metersMerged['END_DATE']).hour

## Created number of days between effective date and end data column
daysDiff = (pd.to_datetime(metersMerged['END_DATE']) - pd.to_datetime(metersMerged['EFFECTIVE_DATE'])).dt.days
metersMerged['DAYS_DIFF'] = daysDiff


## Reordered the columns
new_columns = ["USDP_ID","EQUIPMENT_NUMBER","EFFECTIVE_DATE",
               "START_YEAR","START_MONTH","START_DAY","START_HOUR",
               "END_DATE","END_YEAR","END_MONTH","END_DAY","END_HOUR",
              "DAYS_DIFF","postal code"]

## Renamed some columns
metersMerged = metersMerged.reindex(columns=new_columns)
metersMerged.columns.values[0] = "SERVICEID" 
metersMerged.columns.values[12] = "DAYS_DIFF" 
metersMerged.columns.values[13] = "POST_CODE" 


## Make a space in between the POST_CODE
for i in range(metersMerged["POST_CODE"].values.size):
    if(isinstance(metersMerged["POST_CODE"].values[i] , str)):
        metersMerged["POST_CODE"].values[i] = metersMerged["POST_CODE"].values[i][:3]+" " + metersMerged["POST_CODE"].values[i][-3:]
    else:
        pass
    
## Covert to csv file
metersMerged.to_csv("metersMerged.csv",index=False)


In [ ]:
## Originally wanted to get the LATs and LONs of each individual customer but we drew too much traffic from the server 
## (may need to get the LATs and LONs manually)

# #Installing geopy API
# !pip install geopy
# import geopy as gp
# from geopy.geocoders import Nominatim
# import pgeocode as pgc

# geolocator = Nominatim(user_agent="specify_your_app_name_here")
# nomi = pgc.Nominatim('CA')
# lat = []
# lon = []
# for i in range(metersNew["POSTCODE"].values.size):
#     if(isinstance(metersNew["POSTCODE"].values[i] , str)):
#         location = geolocator.geocode(metersNew["POSTCODE"][i])
        
#         if(location is None):
#             lat.append(nomi.query_postal_code(location).latitude)
#             lon.append(nomi.query_postal_code(location).longitude)

#         else:
#             lat.append(location.latitude)
#             lon.append(location.longitude)   
#     else:
#         pass
# print(i)
# metersNew["LAT"] = lat
# metersNew["LON"] = lon

## Data Preprocessing Step 2 - Finding True Labels
Issue: With multiple datasources the goal was to isolate those datapoints that have true labels. This was done in the following step.

In [4]:
#Import the new mergedMeters.csv data (36095 rows x 14 cols) it will now be called "dfmetersNew"
dfmetersNew = pd.read_csv("metersMerged.csv")


In [5]:
# (1) Find all the serviceIDs that have start year and end year as 2016
all2016 = []
for index, row in dfmetersNew.iterrows():
    if(row["END_YEAR"]==2016 and row["START_YEAR"]==2016):
        all2016.append(row)
        

# (2) Filter out for only instances where there is a postal codes

df2016 = pd.DataFrame(all2016)
postCodeBool = pd.notnull(df2016["POST_CODE"]) 
df2016new = df2016[postCodeBool]
df2016newTrueSID = df2016new["SERVICEID"]


In [ ]:
# (4) Unfortunately, only some of the true label SERVICEIDs exist out of the now 10 avaiable. We extract only the SERVICEIDs that both exist in our 10 avaible and
# in the Service Power Consumption logs

file = pd.read_csv("Sub-Electric-2016-" + str(1)+".csv")
uniqueSID = list(file["SERVICEID"].unique())

uniqueSID = set(uniqueServiceID)
df2016newTrueSID = set(df2016new["SERVICEID"])
availSID = uniqueSID & df2016newTrueSID
availSID = list(availSID)

df2016newprime = df2016new.loc[df2016new['SERVICEID'].isin(availSID)]

# We do the same thing with our created dictionary (getting a subset)
dict_avail =  {}
dict_avail = {key: value for key, value in tempDict.items() for sublist in value if sublist in availSID}


## Feature Engineering Step 3 - (Creating features based on Postal Code)
Now we need more features to increase the complexity of our dataset. Having more features gives us more options in evaluating more combination of features. 
<br> You may choose to drop features that have weak prediction outcome during the validation step.

In [ ]:
# (5) In this step we try to get the power consumption (kwh) for service within the time frame of the faulted serviceID 
# (eg. if my meter was installed in 2016-07-20 and fault occured 2016-7-31 then I would get the power consumption for 11 days, and we do this for all SERVICEIDs
# that have the same postal code as the one that has faulted)
# Create an output file where you are able to get a file filled with kWh

kWHprime = pd.DataFrame({"SERVICEID":[],"START_YEAR":[],"START_MONTH":[],"START_DAY":[],"END_YEAR":[],"END_MONTH":[],"END_DAY":[],
                         "DAY_DIFF":[],"KWH":[], "MAX_KWH":[], "PEAK_DAY_DIFF":[], "POST_CODE":[]})

## This would be every value in the subset of the POST_CODE
for index, row in df2016newprime.iterrows():
    
    countSIDs = 0 
    
    print(row["POST_CODE"])
    print("SERVICEID:",row["SERVICEID"])
    
    sYear = int(row["START_YEAR"])
    sMonth = int(row["START_MONTH"])
    sDay = int(row["START_DAY"])
    
    eYear = int(row["END_YEAR"])
    eMonth = int(row["END_MONTH"])
    eDay = int(row["END_DAY"])
    
    totDaydiff = days_between(sYear, sMonth, sDay, eYear, eMonth, eDay)
    print(totDaydiff)
    
    for value in dict_avail[row["POST_CODE"]]:
        print("ServiceID:",value)
        maxKWH = 0
        midkWH = 0
        #Reading the files
        startfile = "Sub-Electric-2016-" + str(int(sMonth))
        print(startfile)
        startkWhTemp = pd.read_csv(startfile+".csv")
        
        
        endfile = "Sub-Electric-2016-" + str(int(eMonth))
        print(endfile)
        endkWhTemp = pd.read_csv(endfile+".csv")
        
        
        #isolating for specific service ids
        startdfSID = startkWhTemp.loc[startkWhTemp["SERVICEID"] == value]
        enddfSID = endkWhTemp.loc[endkWhTemp["SERVICEID"] == value]
        
        #summing total kwh for the start and end month
        startdfSIDkWH = startdfSID.loc[startdfSID["DAYID"]>=sDay].sum()["KWH"]
        enddfSIDkWH = enddfSID.loc[enddfSID["DAYID"]<=eDay].sum()["KWH"]
        
        #finding the max KWH for the start month
        startdfmaxkWH = startdfSID.loc[startdfSID["DAYID"]>=sDay].max()["KWH"]
        startdfdaykWH = startdfSID.loc[startdfSID["KWH"] == startdfmaxkWH]["DAYID"].iloc[-1] 
        startdfmonthkWH = startdfSID.loc[startdfSID["KWH"] == startdfmaxkWH]["MONTHID"].iloc[-1] 
        startdfyearkWH = startdfSID.loc[startdfSID["KWH"] == startdfmaxkWH]["YEARID"].iloc[-1] 
        
        #finding the max KWH for the end month
        enddfmaxkWHtemp = enddfSID.loc[enddfSID["MONTHID"]<=eMonth]
        enddfmaxkWHtemp = enddfmaxkWHtemp.loc[enddfmaxkWHtemp["DAYID"]<=eDay]
        enddfmaxkWH = enddfmaxkWHtemp.max()["KWH"]
        
        enddfdaykWH = enddfmaxkWHtemp.loc[enddfmaxkWHtemp["KWH"] == enddfmaxkWH]["DAYID"].iloc[-1] 
        enddfmonthkWH = enddfmaxkWHtemp.loc[enddfmaxkWHtemp["KWH"] == enddfmaxkWH]["MONTHID"].iloc[-1] 
        enddfyearkWH = enddfmaxkWHtemp.loc[enddfmaxkWHtemp["KWH"] == enddfmaxkWH]["YEARID"].iloc[-1]

        #finding the max KWH for the start month
        if(startdfmaxkWH > enddfmaxkWH):
            maxKWH = startdfmaxkWH
            maxYr,maxMon,maxDay = startdfyearkWH,startdfmonthkWH,startdfdaykWH
        else:
            maxKWH = enddfmaxkWH
            maxYr,maxMon,maxDay = enddfyearkWH,enddfmonthkWH,enddfdaykWH

        #summing the total kWH for the middle monthes 
        for i in range(int(eMonth)-int(sMonth)-1):
            midfile = "Sub-Electric-2016-" + str(int(sMonth)+i+1)
            print(midfile)
            midkWhTemp = pd.read_csv(midfile+".csv")
            middfSIDkWHs = midkWhTemp.loc[midkWhTemp["SERVICEID"] == value].sum()["KWH"]
            midkWH = midkWH + middfSIDkWHs
            
            middfmaxkWH = midkWhTemp.loc[midkWhTemp["SERVICEID"] == value].max()["KWH"]
            middfdaykWH = midkWhTemp.loc[midkWhTemp["KWH"] == middfmaxkWH]["DAYID"].iloc[-1] 
            middfmonthkWH = midkWhTemp.loc[midkWhTemp["KWH"] == middfmaxkWH]["MONTHID"].iloc[-1] 
            middfyearkWH = midkWhTemp.loc[midkWhTemp["KWH"] == middfmaxkWH]["YEARID"].iloc[-1] 
            
            if (middfmaxkWH>maxKWH):
                maxKWH = middfmaxkWH
                maxYr,maxMon,maxDay = middfyearkWH,middfmonthkWH,middfdaykWH
            else:
                continue
        countSIDs = countSIDs + 1
        totalkWH = startdfSIDkWH + enddfSIDkWH + midkWH
        PeakDaydiff = days_between(sYear, sMonth, sDay, maxYr, maxMon, maxDay)
        totDaydiff = days_between(sYear, sMonth, sDay, eYear, eMonth, eDay)
        kWHprime = kWHprime.append({"SERVICEID":value,"START_YEAR":sYear,"START_MONTH":sMonth,"START_DAY":sDay,"START_YEAR":sYear,
                                    "END_YEAR":eYear,"END_MONTH":eMonth,"END_DAY":eDay, "DAY_DIFF": totDaydiff ,"KWH":totalkWH,"MAX_KWH":maxKWH, 
                                    "PEAK_DAY_DIFF":(totDaydiff-PeakDaydiff), "POST_CODE":row["POST_CODE"]},ignore_index=True)
kWHprime.to_csv("kWHprime.csv")   

In [228]:
# (6) **Note: This is purely based on the assumption that the POST code only failed once in the year. This will be incorrect otherwise

#Get the number of service IDs in a postal Region
kWHprimeNew = kWHprime
kWHprimeUnqiue = kWHprime["POST_CODE"].unique()
templs = []
for postcode in kWHprimeUnqiue:
    templs.append(kWHprime["POST_CODE"].str.count(postcode).sum())
tempData = {"POST_CODE":kWHprimeUnqiue,"POST_COUNT":templs}
tempdf = pd.DataFrame(tempData)

#Get Average KWH for each service ID
kWHprimeNew["AVG_KWH"] = kWHprimeNew["KWH"]/kWHprimeNew["DAY_DIFF"]

#Get Average KWH for each postal region
kWHprimeAvgKWHPost = kWHprimeNew[["AVG_KWH","POST_CODE"]]
kWHprimeAvgKWHPost = kWHprimeAvgKWHPost.groupby(['POST_CODE']).sum()
kWHprimeAvgKWHPost.columns.values[0] = "POST_AVG_KWH" 
kWHprimeAvgKWHPost = pd.merge(tempdf, kWHprimeAvgKWHPost, left_on = 'POST_CODE', right_on = 'POST_CODE', how = 'left')
kWHprimeAvgKWHPost["POST_AVG_KWH"] = kWHprimeAvgKWHPost["POST_AVG_KWH"]/kWHprimeAvgKWHPost["POST_COUNT"]

#Get Total KWH for each postal region
kWHprimeTotKWHPost = kWHprimeAvgKWHPost
kWHprimeTotKWHPost
kWHprimeTotKWHPost = kWHprimeNew[["KWH","POST_CODE"]]
kWHprimeTotKWHPost = kWHprimeTotKWHPost.groupby(['POST_CODE']).sum()
kWHprimeTotKWHPost.columns.values[0] = "POST_TOT_KWH" 
kWHprimeTotKWHPost = pd.merge(kWHprimeAvgKWHPost, kWHprimeTotKWHPost, left_on = 'POST_CODE', right_on = 'POST_CODE', how = 'left')
kWHprimeTotKWHPost["POST_TOT_KWH"] = kWHprimeTotKWHPost["POST_TOT_KWH"]

#Get the Peak KWH for the region.
templs = []
for postcode in kWHprimeUnqiue:
    templs.append(kWHprimeNew[kWHprimeNew['POST_CODE'].isin([postcode])]["MAX_KWH"].max())
tempData = {"POST_CODE":kWHprimeUnqiue,"POST_MAX_KWH":templs}
tempdf = pd.DataFrame(tempData)    
kWHprimeNew = pd.merge(kWHprimeNew, tempdf, left_on = 'POST_CODE', right_on = 'POST_CODE', how = 'left')
kWHprimeNew = pd.merge(kWHprimeNew, kWHprimeTotKWHPost, left_on = 'POST_CODE', right_on = 'POST_CODE', how = 'left')
kWHprimeNew

# Reorder and Rename KWH to TOT_KWH
kWHprimeNew.columns.values[8] = "TOT_KWH" 
kWHprimeNew = kWHprimeNew[kWHprimeNew["SERVICEID"].isin(availSID)]
print(kWHprimeNew.shape) #There is only 2 true values

(2, 17)


## Classification Models Step 4 - Training and Testing the Dataset 
### TO BE CONITNUED BY QMIND 2020-2021
Down below I provided a pipeline and necessary libraries needed to run the machine learning classifier. 
<br> When you are training the model, I would ensure that you drop the DAYS_DIFF features. Based on intution I think this will lead to overfitting.
<br> Also when getting negative (meters that haven't faulted) I would use the default DAYS_DIFF as 1000 and capture the last 1000 days. This is just so you can fill in than putting a null-value

In [233]:
#Imports necessary to run the pipeline function I provided
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import plot_confusion_matrix
from statistics import mean
import seaborn as sns

In [230]:
#Classification pipeline that has 5-fold stratified cross validation
def model(df,classifer,title):
#Splitting up X and Y
    print(df)
    X = df.iloc[:,0:-1]
    X = X.values
    y = df.iloc[:,-1:]
    y = y.values

    listPrecision = []
    listRecall = []
    listAuc = []
    listF1 = []
    listCM = []
    listMCC = []

    countFold = 1

    cv = StratifiedKFold(n_splits=5, random_state = 351)
    for train_idx, test_idx, in cv.split(X,y):    

    print("Running fold:", countFold)
    X_train, X_test = X[train_idx,:], X[test_idx,:] 
    y_train, y_test = y[train_idx,:], y[test_idx,:]

    clf = classifer

    model = Pipeline([('RF', clf)])
    model.set_params(RF__random_state=351).fit(X_train, y_train.ravel())

    y_pred = model.predict(X_test)


    precision = precision_score(y_test, y_pred) #scores for random forest model
    listPrecision.append(round(precision,5))
    recall = recall_score(y_test, y_pred)
    listRecall.append(round(recall,5))
    auc = roc_auc_score(y_test,y_pred)
    listAuc.append(round(auc,5))
    f1 = f1_score(y_test,y_pred)
    listF1.append(round(f1,5))
    mcc = matthews_corrcoef(y_test,y_pred)
    listMCC.append(round(mcc,5))


    cm = confusion_matrix(y_test,y_pred)   
    cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    print('Classification report \n {} \n'.format(classification_report(y_test, y_pred)))
    print("Confusion Matrix: ", cm)

    ax = plt.subplot()
    sns.set(font_scale=2) 
    sns.heatmap(cmn, annot=True, ax=ax, cmap="Blues", fmt=".2g");  


    label_font = {'size':'20'}  
    ax.set_xlabel('Predicted labels', fontdict=label_font);
    ax.set_ylabel('True labels', fontdict=label_font);


    ax.tick_params(axis='both', which='major', labelsize=16) 
    ax.xaxis.set_ticklabels(['Non-PD', 'PD']);
    ax.yaxis.set_ticklabels(['Non-PD', 'PD']);
    plt.show()
    countFold = countFold+1



    print(title + " Precision Score: ",listPrecision , " |Average Precision Score: ", mean(listPrecision))
    print(title + " Recall Score: ",listRecall,  " |Average Recall Score: ", mean(listRecall))
    print(title + " AUC Score: ",listAuc,  " |Average AUC Score: ", mean(listAuc))
    print(title + " F1 Score: ",listF1,  " |Average F1 Score: ", mean(listF1))
    print(title + " MCC Score: ",listMCC,  " |Average MCC Score: ", mean(listMCC))

    return clf

In [ ]:
# Random forest classifer call
clfBL = model(df,RandomForestClassifier(random_state=351),"Random Forest")

## Final Comments
I have provided the necessary files to be able to give you an example of out some of the dataframes should look.
The "KWH_LESS.csv", "KWH_MAX.csv", and "KWH_Zeros.csv" maybe be useful to look at for future analysis